In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from datetime import datetime, date, timedelta
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window
import random
import pandas as pd
import numpy as np
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
import matplotlib.pyplot as plt
import os
import time
del sum

import warnings
warnings.filterwarnings("ignore")


In [2]:
import importlib
import data_preprocess
import label_transactions
import  calculate_persona
import regression_util

In [3]:
### create spark session
spark = SparkSession.\
              builder.\
              appName("ReadParquetFile").getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/05 11:42:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
### read main data
venmo = spark.read.parquet("./data_source/VenmoSample_snappy.parquet")
emoji_dict = pd.read_csv("./data_source/Venmo_Emoji_Classification_Dictionary.csv")
word_dict = pd.read_csv("./data_source/Venmo_Word_Classification_Dictionary.csv")

In [6]:
### preprocess the data
venmo_df, emoji_df, word_df = data_preprocess.data_preprocess(venmo, [emoji_dict, word_dict], spark)

### map the transactions
matched_df = label_transactions.label_transactions(venmo_df, word_df, emoji_df, spark, save_data = True)

In [5]:
### calcualte the persona based on the need
final_df_for_regression1 = calculate_persona.calculate_persona(matched_df, spark, start_lifetime_month = 1, end_lifetime_month = 6, save_data = False)

stage : start user combine.......complete
stage : start calculating lifetime month.......complete
stage : start joining friends transactions.......complete
stage : start calculating user persona.......

complete
stage : start calculating friends persona.......

complete


In [6]:
### calcualte the persona based on the need
final_df_for_regression2 = calculate_persona.calculate_persona(matched_df, spark, start_lifetime_month = 6, end_lifetime_month = 9, save_data = False)

stage : start user combine.......complete
stage : start calculating lifetime month.......complete
stage : start joining friends transactions.......complete
stage : start calculating user persona.......

complete


In [7]:
final_df_for_regression1.show()

+-------+--------+----+-----+----+------------------+------+------------------+------+-------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+-------------------+
|   user|Activity|Cash|Event|Food|   Illegal/Sarcasm|People|    Transportation|Travel|Utility|         c_Activity|             c_Cash|            c_Event|             c_Food|   c_Illegal/Sarcasm|           c_People|   c_Transportation|           c_Travel|          c_Utility|
+-------+--------+----+-----+----+------------------+------+------------------+------+-------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+-------------------+
|     31|     0.0| 0.0|  0.0| 1.0|               0.0|   0.0|               0.0|   0.0|    0.0|                0.0|                0.0|                0.0|                0.

In [8]:
final_df_for_regression2.show()

+-------+------------------+----+-----+------------------+---------------+------------------+------------------+------+------------------+------------------+------+-------+------------------+-----------------+--------+------------------+--------+---------+
|   user|          Activity|Cash|Event|              Food|Illegal/Sarcasm|            People|    Transportation|Travel|           Utility|        c_Activity|c_Cash|c_Event|            c_Food|c_Illegal/Sarcasm|c_People|  c_Transportation|c_Travel|c_Utility|
+-------+------------------+----+-----+------------------+---------------+------------------+------------------+------+------------------+------------------+------+-------+------------------+-----------------+--------+------------------+--------+---------+
|     31|               0.0| 0.0|  0.0|               1.0|            0.0|               0.0|               0.0|   0.0|               0.0|               0.0|   0.0|    0.0|               0.0|              0.0|     0.0|           

In [5]:
final_df_for_regression1 = spark.read.parquet("./output/final_df_for_regression1.parquet")

In [6]:
### regress the variables on the target
### 1. regress the community persona on user persona
### data preprocess for regression
train1, test1 = regression_util.preprocess_community_model_data(final_df_for_regression1, spark)

### run the model
model_list1, result_df1 = regression_util.train_regression(train1, spark)

### make the forecast
predicted_df1 = regression_util.make_forecast(model_list1, test1, spark)

### evaluate the forecast
rmse1, calcualte_df1 = regression_util.calculate_rmse(predicted_df1, spark)

stage : start training label_activity.......

complete
stage : start training label_Cash.......

complete
stage : start training label_Event.......

complete
stage : start training label_Food.......

complete
stage : start training label_Illegal/Sarcasm.......

complete
stage : start training label_People.......

complete
stage : start training label_Transportation.......

complete
stage : start training label_Travel.......

complete
stage : start training label_Utility.......

complete


In [7]:
print(f'The RMSE for the first model: {rmse1}')

The RMSE for the first model: 0.2031965343569859


In [9]:
final_df_for_regression2 = spark.read.parquet("./output/final_df_for_regression2.parquet")

In [10]:
### 2. regress the user persona for the first 6 months on the user persona of the next month
### data preprocess for regression
train2, test2 = regression_util.preprocess_community_model_data(final_df_for_regression2, spark)

### run the model
model_list2, result_df2 = regression_util.train_regression(train2, spark)

### make the forecast
predicted_df2 = regression_util.make_forecast(model_list2, test2, spark)

### evaluate the forecast
rmse2, calcualte_df2 = regression_util.calculate_rmse(predicted_df2, spark)

stage : start training label_activity.......

complete
stage : start training label_Cash.......

complete
stage : start training label_Event.......

complete
stage : start training label_Food.......

complete
stage : start training label_Illegal/Sarcasm.......

complete
stage : start training label_People.......

complete
stage : start training label_Transportation.......

complete
stage : start training label_Travel.......

complete
stage : start training label_Utility.......

complete


In [11]:
print(f'The RMSE for the second model: {rmse2}')

The RMSE for the second model: 0.2751961379954288


### Further analysis

In [7]:
### read main data
matched_df = spark.read.parquet("./output/matched_df.parquet/")
matched_df.show()

+--------------------+------+------+-----------------+--------------------+-------------------+-----------+---------------+-----------+----------+----------+-------------------+
|            story_id| user1| user2|transactioni_type|         description|   transaction_time|emoji_label|     word_label|emoji_token|word_token|match_type|selected_match_type|
+--------------------+------+------+-----------------+--------------------+-------------------+-----------+---------------+-----------+----------+----------+-------------------+
|4f869e4c33279da6c...| 90144| 84558|          payment|;for;money;resurf...|2012-04-12 09:20:12|       null|           Cash|       null|     money| word only|               Cash|
|4f888f3b33279da6c...| 82682| 83682|           charge|;for;all;sorts;of...|2012-04-13 20:40:26|       null|         People|       null|      cool| word only|             People|
|4f8de1ff33279da6c...| 88265| 88285|          payment|;for;tasty;oz;lun...|2012-04-17 21:34:54|       null|   

In [9]:
### create a view for dataframes
matched_df.createOrReplaceTempView('matched_df')

In [13]:
### count word token
word_count_df = spark.sql("""
SELECT word_token, COUNT(story_id) AS count
FROM matched_df
WHERE word_token IS NOT NULL AND word_label = 'Food'
GROUP BY word_token
ORDER BY count DESC, word_token
""")

word_count_df.show()

+----------+------+
|word_token| count|
+----------+------+
|      food|107087|
|    dinner| 32858|
|     pizza| 29221|
|     water| 22885|
| groceries| 22086|
|     lunch| 19701|
|     salad| 15136|
|   chicken| 11921|
|    costco| 11008|
|       tip| 10963|
|      thai| 10693|
|    brunch| 10492|
|        za| 10084|
|      wine|  9241|
|       bbq|  9030|
|     booze|  8976|
|     drank|  8592|
|       hot|  8486|
|     sushi|  8226|
| breakfast|  7840|
+----------+------+
only showing top 20 rows



In [14]:
### count emoji token
emoji_count_df = spark.sql("""
SELECT emoji_token, COUNT(story_id) AS count
FROM matched_df
WHERE emoji_token IS NOT NULL AND emoji_label = 'Food'
GROUP BY emoji_token
ORDER BY count DESC, emoji_token
""")

emoji_count_df.show()

+-----------+-----+
|emoji_token|count|
+-----------+-----+
|         🍕|95879|
|         🍻|70328|
|         🍷|49356|
|         🍺|42944|
|         🍴|39347|
|         🍔|33873|
|         🍾|31965|
|          ☕|27827|
|         🍹|24880|
|         🍜|23045|
|         🌮|22495|
|         🍣|22353|
|         🍳|22256|
|         🍸|20806|
|         🍗|19953|
|         🍆|14831|
|         🍞|12629|
|         🍲|12515|
|         🍦|11681|
|         🍟|11499|
+-----------+-----+
only showing top 20 rows



23/06/05 23:00:11 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:113)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:112)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:548)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:547)
	at org.apache.spar